In [18]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

import punch_party_utils

In [2]:
average_air_quality = punch_party_utils.make_average_aq_df()
average_air_quality = average_air_quality[average_air_quality["MeasureName"] == "Annual average ambient concentrations of PM2.5 in micrograms per cubic meter (based on seasonal averages and daily measurement)"]

In [3]:
# For the first model, just CO2 emmisions in a given year -> air quality in a given year
average_air_quality = average_air_quality.drop(
    columns=["MeasureId", "MeasureName", "MeasureType", "StratificationLevel",
             "StateFips", "StateName", "CountyFips", "CountyName", "Unit",
             "UnitName", "DataOrigin", "MonitorOnly"])

display(average_air_quality.columns)


Index(['ReportYear', 'Value'], dtype='object')

In [4]:
print(len(average_air_quality))

aq_by_year = average_air_quality.groupby("ReportYear")

8422


In [5]:
air_quality_yearly_averages = aq_by_year.agg("mean")
print(air_quality_yearly_averages)

                Value
ReportYear           
1999        13.748994
2000        13.020075
2001        12.403782
2002        11.956124
2003        11.808624
2004        11.550947
2005        12.607519
2006        11.432796
2007        11.772843
2008        10.688418
2009         9.528830
2010         9.790810
2011         9.672332
2012         9.063382
2013         8.821925


In [6]:
gas_types = punch_party_utils.make_gas_types_df()

In [7]:
display(gas_types.columns)

gas_types = gas_types.drop(
    columns=["Address Line 1", "City", "County", 
             "Facility ID", "Gas Code", "Latitude", "Longitude", 
             "State Name", "Zip Code", "Facility Name"])

Index(['Address Line 1', 'City', 'CO2 Emission', 'County', 'Facility ID',
       'Gas Code', 'Gas Name', 'Latitude', 'Longitude', 'State Name', 'Year',
       'Zip Code', 'Facility Name'],
      dtype='object')

In [8]:
display(gas_types.head())

,CO2 Emission,Gas Name,Year
0,58024.000,BIOGENIC CO2,2017
1,134.500,METHANE,2018
2,6.854,NITROUS OXIDE,2012
3,53562.000,METHANE,2017
4,7635064.700,CARBON DIOXIDE,2018


In [10]:
raw_emmisions_only = gas_types.drop(columns=["Gas Name"])
yearly_raw_emmisions = raw_emmisions_only.groupby("Year").agg("sum")

In [11]:
print(yearly_raw_emmisions)

      CO2 Emission
Year              
2010  3.229325e+09
2011  3.333042e+09
2012  3.194602e+09
2013  3.210963e+09
2014  3.222917e+09
2015  3.078839e+09
2016  3.019157e+09
2017  2.947673e+09
2018  3.037238e+09
2019  2.898178e+09


In [12]:
co2_emmisions_with_aq = yearly_raw_emmisions.merge(
    air_quality_yearly_averages, left_index=True, right_index=True, how="inner")
print(co2_emmisions_with_aq)

      CO2 Emission     Value
2010  3.229325e+09  9.790810
2011  3.333042e+09  9.672332
2012  3.194602e+09  9.063382
2013  3.210963e+09  8.821925


In [13]:
# Normalize
max_co2_emission = np.max(co2_emmisions_with_aq["CO2 Emission"])
max_co2_emission
co2_emmisions_with_aq["CO2 Emission"] = co2_emmisions_with_aq["CO2 Emission"] / max_co2_emission

display(co2_emmisions_with_aq)

,CO2 Emission,Value
2010,0.968882,9.790810
2011,1.000000,9.672332
2012,0.958464,9.063382
2013,0.963373,8.821925


In [14]:
co2_aq_model = LinearRegression(fit_intercept=True)

independent_var = co2_emmisions_with_aq[["CO2 Emission"]]
dependent_var = co2_emmisions_with_aq["Value"]

display(independent_var)
display(dependent_var)

co2_aq_model.fit(independent_var, dependent_var)

,CO2 Emission
2010,0.968882
2011,1.000000
2012,0.958464
2013,0.963373


2010    9.790810
2011    9.672332
2012    9.063382
2013    8.821925
Name: Value, dtype: float64

LinearRegression()

In [15]:
# Divide value to predict by max emission from training for normalization consistency
value_to_predict = (3.2 * 10**9) / max_co2_emission

print(co2_aq_model.predict(np.array([value_to_predict]).reshape(-1, 1)))

0.9600838993850696

3333042041.4815717

[9.14362915]


/home/sean/miniconda3/envs/msse-python/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [35]:
emissions_per_year_per_gas = gas_types.groupby(['Year', 'Gas Name']).mean()

emissions_per_year_per_gas = emissions_per_year_per_gas.reset_index()

emissions_per_year_per_gas = emissions_per_year_per_gas.pivot(index='Year', columns='Gas Name', values='CO2 Emission').dropna()

display(emissions_per_year_per_gas)

Gas Name,BIOGENIC CO2,CARBON DIOXIDE,HFCS,HFES,METHANE,NITROGEN TRIFLOURIDE,NITROUS OXIDE,OTHER,OTHER FULLY FLUORINATED GHGS,PFCS,SULFUR HEXAFLUORIDE,VERY SHORT-LIVED COMPOUNDS
Year,,,,,,,,,,,,
2011,219854.886983,462302.634066,196118.872666,888.608834,31507.831677,22648.793747,6002.174766,6747.611722,28001.296085,113693.707661,34794.461450,7.803089
2012,221692.959997,428138.510385,153791.590090,680.194925,29956.920238,20703.412830,4755.363164,6425.133333,28315.847044,104668.101817,28516.478182,5.895594
2013,215409.963821,427731.424518,129820.451666,623.485861,29092.144228,16215.268148,4291.312665,5294.825375,32481.755893,103245.245113,27478.328136,8.221518
2014,211993.601611,420014.630570,121274.558525,1409.273671,28052.491027,15063.658824,4523.518900,4466.333333,27371.955091,93473.825614,27592.388072,13.654441
2015,214321.305711,405575.849913,100378.183108,4111.709119,28259.379143,16324.279200,4282.749662,4692.033333,25813.783808,87343.614173,24874.326846,10.037224
2016,222358.176970,400135.672860,71522.700426,4796.704073,30815.509263,15849.905306,4473.979602,5659.666667,20543.399321,80593.112847,32661.720684,9.980153
2017,233888.923987,395373.907515,108830.924316,2251.545667,30037.760949,21363.596522,4437.589175,7407.837500,19329.803654,79049.098555,30441.777313,175.140414
2018,236278.276721,402524.464243,85099.096264,2114.205038,29896.962297,15969.512000,4834.939194,12032.225000,20636.510176,92902.147146,29139.825000,17.776397
2019,245855.440780,383535.103409,86878.502970,2091.567750,30272.276821,15695.917333,3913.113277,9776.488889,21070.790851,96507.384590,30307.531385,6.964801


In [37]:
all_gasses_with_air_quality = emissions_per_year_per_gas.merge(
    air_quality_yearly_averages, left_index=True, right_index=True, how="inner")
display(all_gasses_with_air_quality) 

,BIOGENIC CO2,CARBON DIOXIDE,HFCS,HFES,METHANE,NITROGEN TRIFLOURIDE,NITROUS OXIDE,OTHER,OTHER FULLY FLUORINATED GHGS,PFCS,SULFUR HEXAFLUORIDE,VERY SHORT-LIVED COMPOUNDS,Value
2011,219854.886983,462302.634066,196118.872666,888.608834,31507.831677,22648.793747,6002.174766,6747.611722,28001.296085,113693.707661,34794.461450,7.803089,9.672332
2012,221692.959997,428138.510385,153791.590090,680.194925,29956.920238,20703.412830,4755.363164,6425.133333,28315.847044,104668.101817,28516.478182,5.895594,9.063382
2013,215409.963821,427731.424518,129820.451666,623.485861,29092.144228,16215.268148,4291.312665,5294.825375,32481.755893,103245.245113,27478.328136,8.221518,8.821925


In [67]:
independent_variables = all_gasses_with_air_quality.drop(columns=["Value"])
dependedent_variables = all_gasses_with_air_quality["Value"]
max_gas_emissions = independent_variables.max()

normalized_max_gas_emmisions = independent_variables / max_gas_emissions
display(normalized_max_gas_emmisions)

<class 'pandas.core.series.Series'>


,BIOGENIC CO2,CARBON DIOXIDE,HFCS,HFES,METHANE,NITROGEN TRIFLOURIDE,NITROUS OXIDE,OTHER,OTHER FULLY FLUORINATED GHGS,PFCS,SULFUR HEXAFLUORIDE,VERY SHORT-LIVED COMPOUNDS
2011,0.991709,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.862062,1.000000,1.000000,0.949106
2012,1.000000,0.92610,0.784175,0.765460,0.950777,0.914107,0.792273,0.952209,0.871746,0.920615,0.819569,0.717093
2013,0.971659,0.92522,0.661948,0.701643,0.923331,0.715944,0.714960,0.784696,1.000000,0.908100,0.789733,1.000000


In [62]:
model_with_gas_types = LinearRegression(fit_intercept=True)

model_with_gas_types.fit(normalized_max_gas_emmisions, dependedent_variables)

print(model_with_gas_types.coef_)

[-0.19696469  0.14554781  0.39970511  0.94732416  0.36555461  0.4516192
  0.47818525  0.23628307 -0.12506562  0.31421171  0.22747971  0.23059716]


In [72]:
example_value_to_predict = np.array([215409.963821, 427731.424518, 129820.451666, 623.485861, 29092.144228, 16215.268148, 4291.312665, 5294.825375, 32481.755893, 103245.245113, 27478.328136, 8.221518]).reshape(-1, 1)

normalized_values_to_predict = example_value_to_predict / np.array(max_gas_emissions).reshape(-1, 1)
normalized_values_to_predict = normalized_values_to_predict.T

prediction = model_with_gas_types.predict(normalized_values_to_predict)
print(prediction)
print(model_with_gas_types.coef_)

[8.82192483]


/home/sean/miniconda3/envs/msse-python/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
